# Recreating the GFOLD paper

Reference: https://arc.aiaa.org/action/doSearch?AllField=Convex+Programming+Approach+to+Powered+Descent+Guidance+for+Mars+Landing

In [82]:
%matplotlib inline
import numpy as np
from scipy.linalg import expm
import matplotlib.pyplot as plt
import cvxpy as cvx

## Powered Descent Guidance Example

In [31]:
ge = 9.807
g = np.array([[-3.711400]])  # m/s^2
mdry = 1505  # kg
mwet = 1905  # kg
mfuel = mwet-mdry
Isp = 225  # s
Tbar = 3100  # N (maximum thrust for each thruster)
T1 = 0.3*Tbar
T2 = 0.8*Tbar
n = 6
phi = 27  # deg
alpha = 1/(Isp*ge*np.cos(np.deg2rad(phi)))

An additional constraint on the thrust direction at time $t_f$ is imposed to be the opposite direction to the gravity vector.

In [4]:
nhatf = np.array([[1, 0, 0]]).T

In [116]:
r0 = np.array([1.5, 0, 2])
rdot0 = np.array([ -75, 0, 100])

y0 = np.append(np.append(r0, rdot0), np.array([np.log(mwet)]))

tmin = (mwet-mfuel)*np.linalg.norm(rdot0)/T2
tmax = (mwet-mfuel)*np.linalg.norm(rdot0)/T1

dt = 1  # s
tf = 72  # s
N = int(tf/dt)
time = np.zeros((0,tf,N))

# initial conditions
ns = 6
state = np.zeros((ns,1,N))  # state is R3 position and velocity
state[:,0,0] = np.append(r0,rdot0)

In [143]:
Ac = np.array([[0, 0, 0, 1, 0, 0, 0],
               [0, 0, 0, 0, 1, 0, 0],
               [0, 0, 0, 0, 0, 1, 0],
               [0, 0, 0, 0, 0, 0, 0],
               [0, 0, 0, 0, 0, 0, 0],
               [0, 0, 0, 0, 0, 0, 0],
               [0, 0, 0, 0, 0, 0, 0]])

Bc = np.array([[0, 0, 0, 0],
               [0, 0, 0, 0],
               [0, 0, 0, 0],
               [1, 0, 0, 0],
               [0, 1, 0, 0],
               [0, 0, 1, 0],
               [0, 0, 0, 0]])

A = expm(Ac*dt)
B = np.matmul(np.eye(len(Ac)) - A,Bc)

In [145]:
Phi = np.zeros((len(A),len(A),N))
dim1, dim2 = B.shape
Lambda = np.zeros((dim1,dim2,N))
for k in range(0,N):
    Phi[:,:,k] = np.linalg.matrix_power(A,k)
    for j in range(0,k):
        Lambda[:,:,k] += np.matmul(np.linalg.matrix_power(A,j),B)

In [148]:
E = np.append(np.eye(6), np.zeros((6,1)),axis=1)
F = np.append(np.zeros((1,6)),np.eye(1))

esigma = np.array([[0,0,0,1]]).T
w = np.ones((1,N))*dt
omega = w
Eu = np.append(np.eye(3), np.zeros((3,1)),axis=1)

In [149]:
w

array([[1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1.]])

In [131]:
np.zeros((3,1))

array([[0.],
       [0.],
       [0.]])